# Battle of Neighborhoods
In this notebook, I will be creating cluster to find a suitable location to open a Pizza Place in Staten Island,New York.

Import required libraries.

In [1]:
import pandas as pd
import numpy as np
import folium
import json
import requests
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans

In [2]:
#Download the dataset.
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [3]:
with open ('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
neighbor_data = newyork_data['features']
neighbor_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

#### Creating a dataframe.

In [4]:
column_name = ['Borough','Neighborhood', 'Latitude', 'Longitude']
neighbors = pd.DataFrame(columns = column_name)
neighbors

,Borough,Neighborhood,Latitude,Longitude


In [5]:
for data in neighbor_data:
    borough = neighbor_name = data['properties']['borough'] 
    neighbor_name = data['properties']['name']
        
    neighbor_latlon = data['geometry']['coordinates']
    neighbor_lat = neighbor_latlon[1]
    neighbor_lon = neighbor_latlon[0]
    
    neighbors = neighbors.append({'Borough': borough,
                                          'Neighborhood': neighbor_name,
                                          'Latitude': neighbor_lat,
                                          'Longitude': neighbor_lon}, ignore_index=True)

In [6]:
neighbors.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [7]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(len(neighbors['Borough'].unique()), neighbors.shape[0]))

The dataframe has 5 boroughs and 306 neighborhoods.


#### Get latitude and longitude of New York. 

In [8]:
address = 'New York City, NY'
geolocator = Nominatim(user_agent = 'ny_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The co-ordinates of new york city are {}, {}".format(latitude,longitude))

The co-ordinates of new york city are 40.7308619, -73.9871558


#### Creating map of New York using Folium.

In [9]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(neighbors['Latitude'], neighbors['Longitude'], neighbors['Borough'], neighbors['Neighborhood']):
    label = '{} {}'.format(neighborhood,borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
                        [lat,lng],
                        radius = 5,
                        popup = label,
                        color = 'red',
                        fill = True,
                        fill_color = '#FF6347',
                        fill_opacity = 0.7,
                        parse_html = False).add_to(map_newyork)
map_newyork    

##### For this project I am using Staten Island neighborhoods. 

Create new dataframe staten_data.

In [10]:
staten_data = neighbors[neighbors["Borough"] == "Staten Island"].reset_index(drop = True)
print(staten_data.shape)
staten_data.head()

(63, 4)


,Borough,Neighborhood,Latitude,Longitude
0,Staten Island,St. George,40.644982,-74.079353
1,Staten Island,New Brighton,40.640615,-74.087017
2,Staten Island,Stapleton,40.626928,-74.077902
3,Staten Island,Rosebank,40.615305,-74.069805
4,Staten Island,West Brighton,40.631879,-74.107182


### Create map of Staten Island.

In [11]:
address = 'Staten Island, NY'
geolocator = Nominatim(user_agent = 'ny_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_staten = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng,label in zip(staten_data['Latitude'], staten_data['Longitude'], staten_data['Neighborhood']):
    label = '{} {}'.format(neighborhood,borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
                        [lat,lng],
                        radius = 5,
                        popup = label,
                        color = 'red',
                        fill = True,
                        fill_color = '#FF6347',
                        fill_opacity = 0.7,
                        parse_html = False).add_to(map_staten)
map_staten  

#### Get nearby venues using Foursquare.

In [19]:
CLIENT_ID = '**********************************' # your Foursquare ID
CLIENT_SECRET = '******************************' # your Foursquare Secret
VERSION = '******' # Foursquare API version
LIMIT = 100

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
staten_venues = getNearbyVenues(names = staten_data['Neighborhood'],
                                latitudes = staten_data['Latitude'],
                                longitudes = staten_data['Longitude'])

St. George
New Brighton
Stapleton
Rosebank
West Brighton
Grymes Hill
Todt Hill
South Beach
Port Richmond
Mariner's Harbor
Port Ivory
Castleton Corners
New Springville
Travis
New Dorp
Oakwood
Great Kills
Eltingville
Annadale
Woodrow
Tottenville
Tompkinsville
Silver Lake
Sunnyside
Park Hill
Westerleigh
Graniteville
Arlington
Arrochar
Grasmere
Old Town
Dongan Hills
Midland Beach
Grant City
New Dorp Beach
Bay Terrace
Huguenot
Pleasant Plains
Butler Manor
Charleston
Rossville
Arden Heights
Greenridge
Heartland Village
Chelsea
Bloomfield
Bulls Head
Richmond Town
Shore Acres
Clifton
Concord
Emerson Hill
Randall Manor
Howland Hook
Elm Park
Manor Heights
Willowbrook
Sandy Ground
Egbertville
Prince's Bay
Lighthouse Hill
Richmond Valley
Fox Hills


In [22]:
print(staten_venues.shape)
staten_venues.head()

(797, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,St. George,40.644982,-74.079353,Richmond County Bank Ballpark,40.645056,-74.076864,Baseball Stadium
1,St. George,40.644982,-74.079353,A&S Pizzeria,40.643940,-74.077626,Pizza Place
2,St. George,40.644982,-74.079353,Staten Island September 11 Memorial,40.646767,-74.076510,Monument / Landmark
3,St. George,40.644982,-74.079353,Beso,40.643306,-74.076508,Tapas Restaurant
4,St. George,40.644982,-74.079353,St. George Theatre,40.642253,-74.077496,Theater


In [23]:
staten_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Annadale,13,13,13,13,13,13
Arden Heights,5,5,5,5,5,5
Arlington,4,4,4,4,4,4
Arrochar,20,20,20,20,20,20
Bay Terrace,11,11,11,11,11,11
Bloomfield,4,4,4,4,4,4
Bulls Head,44,44,44,44,44,44
Butler Manor,6,6,6,6,6,6
Castleton Corners,14,14,14,14,14,14


In [24]:
print('There are {} uniques categories.'.format(len(staten_venues['Venue Category'].unique())))

There are 168 uniques categories.


In [25]:
staten_venues['Venue Category'].unique()

array(['Baseball Stadium', 'Pizza Place', 'Monument / Landmark',
       'Tapas Restaurant', 'Theater', 'Italian Restaurant', 'Bar',
       'American Restaurant', 'Toy / Game Store', 'Art Gallery',
       'Donut Shop', 'Steakhouse', 'Plaza', 'Harbor / Marina',
       'Scenic Lookout', 'Park', 'Bus Stop', 'Tourist Information Center',
       'Discount Store', 'Bowling Alley', 'Deli / Bodega', 'Playground',
       'Convenience Store', 'Chinese Restaurant', 'Laundromat',
       'Sandwich Place', 'Sri Lankan Restaurant', 'Restaurant',
       'Spanish Restaurant', 'Asian Restaurant', 'Beer Bar', 'Skate Park',
       'Residential Building (Apartment / Condo)', 'Mexican Restaurant',
       'New American Restaurant', 'Bank', 'Fast Food Restaurant',
       'Cosmetics Shop', 'Train Station', 'Pool Hall', 'Coffee Shop',
       'Cajun / Creole Restaurant', 'Breakfast Spot',
       'Filipino Restaurant', 'Grocery Store',
       'Eastern European Restaurant', 'Pharmacy', 'Bagel Shop',
       'Video S

#### Analyze each neighborhood.

In [26]:
#One hot encoding
staten_onehot = pd.get_dummies(staten_venues[['Venue Category']], prefix = "", prefix_sep = "")
staten_onehot['Neighborhood'] = staten_venues['Neighborhood']
fixed_column = [staten_onehot.columns[-1]] + list(staten_onehot.columns[:-1])
staten_onehot = staten_onehot[fixed_column]
staten_onehot.head()

,Neighborhood,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,...,Tourist Information Center,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,St. George,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,St. George,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,St. George,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,St. George,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,St. George,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
staten_onehot["Neighborhood"].head()

0    St. George
1    St. George
2    St. George
3    St. George
4    St. George
Name: Neighborhood, dtype: object

In [28]:
staten_grp = staten_onehot.groupby(["Neighborhood"]).mean().reset_index()
print(staten_grp.shape)
staten_grp.head()

(62, 169)


,Neighborhood,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,...,Tourist Information Center,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Annadale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.076923,0.0,0.0,0.0,0.0,0.0,0.0
1,Arden Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,Arlington,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,Arrochar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,Bay Terrace,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.090909,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
len(staten_grp[staten_grp["Pizza Place"]>0])

29

#### Creating new dataframe containing only Diner.

In [30]:
pp_df = staten_grp[["Neighborhood","Pizza Place"]]
print(pp_df.shape)
pp_df.head()

(62, 2)


,Neighborhood,Pizza Place
0,Annadale,0.230769
1,Arden Heights,0.200000
2,Arlington,0.000000
3,Arrochar,0.100000
4,Bay Terrace,0.000000


### Clustering Neighborhoods using K-Means.

In [31]:
k_clusters = 3
staten_cluster = pp_df.drop(["Neighborhood"],1)

kmeans = KMeans(n_clusters = k_clusters, random_state = 0).fit(staten_cluster)
kmeans.labels_[0:]

array([2, 2, 0, 1, 0, 0, 1, 0, 2, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       1, 2, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 2, 2, 0, 0,
       0, 1, 2, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1], dtype=int32)

In [32]:
pp_cpy = pp_df.copy()
pp_cpy["Cluster Labels"] = kmeans.labels_
pp_cpy.rename(columns = {'Neighborhood':'Neighborhood'},inplace = True)
pp_cpy.head()

,Neighborhood,Pizza Place,Cluster Labels
0,Annadale,0.230769,2
1,Arden Heights,0.200000,2
2,Arlington,0.000000,0
3,Arrochar,0.100000,1
4,Bay Terrace,0.000000,0


In [33]:
pp_cpy = pp_cpy.join(staten_venues.set_index("Neighborhood"), on = "Neighborhood")
print(pp_cpy.shape)
pp_cpy.head()

(797, 9)


,Neighborhood,Pizza Place,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Annadale,0.230769,2,40.538114,-74.178549,Play Sports Bar,40.540418,-74.177196,Sports Bar
0,Annadale,0.230769,2,40.538114,-74.178549,Annadale Diner,40.542079,-74.177325,Diner
0,Annadale,0.230769,2,40.538114,-74.178549,Il Sogno,40.541286,-74.178489,Restaurant
0,Annadale,0.230769,2,40.538114,-74.178549,MTA SIR - Annadale,40.540482,-74.178185,Train Station
0,Annadale,0.230769,2,40.538114,-74.178549,Amy's Pet Palace,40.539992,-74.176734,Pet Store


In [34]:
# Sorting the result by cluster labels.
pp_cpy.sort_values(["Cluster Labels"],inplace = True)
pp_cpy.head()

,Neighborhood,Pizza Place,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
30,Midland Beach,0.000000,0,40.573527,-74.093483,Laroccas Italian Ices and Pizzeria,40.574205,-74.096310,Dessert Shop
48,Shore Acres,0.043478,0,40.609719,-74.066678,Fingerboard Rd & Tompkins Ave,40.606976,-74.067131,Intersection
48,Shore Acres,0.043478,0,40.609719,-74.066678,MTA Bus - Bay St & New La (S51),40.611977,-74.064547,Bus Stop
48,Shore Acres,0.043478,0,40.609719,-74.066678,Phrygian Studios,40.613027,-74.065413,Music Store
48,Shore Acres,0.043478,0,40.609719,-74.066678,East Shore LL,40.606586,-74.063872,Baseball Field


### Visualizing the Clusters.

In [35]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [36]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(pp_cpy['Neighborhood Latitude'], pp_cpy['Neighborhood Longitude'], pp_cpy['Neighborhood'],pp_cpy['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [37]:
#Save the map .
map_clusters.save("map_clusters.png")

In [31]:
#Cluster 0
cluster_0 = pp_cpy.loc[pp_cpy["Cluster Labels"]==0]
cluster_0

,Neighborhood,Pizza Place,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
30,Midland Beach,0.000000,0,40.573527,-74.093483,Laroccas Italian Ices and Pizzeria,40.574205,-74.096310,Dessert Shop
48,Shore Acres,0.043478,0,40.609719,-74.066678,Fingerboard Rd & Tompkins Ave,40.606976,-74.067131,Intersection
48,Shore Acres,0.043478,0,40.609719,-74.066678,MTA Bus - Bay St & New La (S51),40.611977,-74.064547,Bus Stop
48,Shore Acres,0.043478,0,40.609719,-74.066678,Phrygian Studios,40.613027,-74.065413,Music Store
48,Shore Acres,0.043478,0,40.609719,-74.066678,East Shore LL,40.606586,-74.063872,Baseball Field
48,Shore Acres,0.043478,0,40.609719,-74.066678,MTA Bus - Hylan Bl & Vaughn St (S78),40.611289,-74.071761,Bus Stop
48,Shore Acres,0.043478,0,40.609719,-74.066678,Marketfresh supermarket,40.611939,-74.071342,Supermarket
48,Shore Acres,0.043478,0,40.609719,-74.066678,Lucky Deli & Grocery,40.613972,-74.066033,Deli / Bodega
48,Shore Acres,0.043478,0,40.609719,-74.066678,Ocean Nail & Spa,40.612459,-74.071022,Nail Salon
17,Emerson Hill,0.000000,0,40.606794,-74.097762,Corleone Mansion (Movie Set),40.606622,-74.097955,Historic Site


In [32]:
#Cluster 1
cluster_1=pp_cpy.loc[pp_cpy["Cluster Labels"]==1]
cluster_1.head()

,Neighborhood,Pizza Place,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
36,Old Town,0.052632,1,40.596329,-74.087511,BP,40.593896,-74.086542,Gas Station
36,Old Town,0.052632,1,40.596329,-74.087511,Dunkin Donuts,40.596585,-74.085241,Donut Shop
36,Old Town,0.052632,1,40.596329,-74.087511,Jimmy's Liquors,40.594710,-74.086773,Liquor Store
36,Old Town,0.052632,1,40.596329,-74.087511,Perkins Restaurant & Bakery,40.594421,-74.087000,American Restaurant
36,Old Town,0.052632,1,40.596329,-74.087511,Domino's Pizza,40.596539,-74.085089,Pizza Place


In [33]:
#Cluster 2
cluster_2 = pp_cpy.loc[pp_cpy["Cluster Labels"]==2]
cluster_2

,Neighborhood,Pizza Place,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1,Arden Heights,0.200000,2,40.549286,-74.185887,CVS pharmacy,40.551241,-74.184031,Pharmacy
1,Arden Heights,0.200000,2,40.549286,-74.185887,APlus at Sunoco,40.551233,-74.184030,Coffee Shop
1,Arden Heights,0.200000,2,40.549286,-74.185887,MTA Bus - Arden Av & Woodrow Rd (S56/X17/X17J/...,40.550082,-74.187375,Bus Stop
1,Arden Heights,0.200000,2,40.549286,-74.185887,Rocco Deli,40.551221,-74.184073,Deli / Bodega
0,Annadale,0.230769,2,40.538114,-74.178549,Amy's Pet Palace,40.539992,-74.176734,Pet Store
1,Arden Heights,0.200000,2,40.549286,-74.185887,Villa Monte Pizzeria & Restaurant,40.552555,-74.186212,Pizza Place
0,Annadale,0.230769,2,40.538114,-74.178549,Major League Liquors,40.541547,-74.175667,Liquor Store
0,Annadale,0.230769,2,40.538114,-74.178549,Annadale Family Pharmacy,40.541611,-74.177102,Pharmacy
0,Annadale,0.230769,2,40.538114,-74.178549,Square Pizza,40.541462,-74.177444,Pizza Place
0,Annadale,0.230769,2,40.538114,-74.178549,Mona Lisa Pizzeria,40.541289,-74.176993,Pizza Place


In [34]:
cluster_0["Neighborhood"].unique()

array(['Midland Beach', 'Shore Acres', 'Emerson Hill', 'Silver Lake',
       'South Beach', 'Fox Hills', 'Graniteville', 'Park Hill',
       'New Dorp Beach', 'Egbertville', 'Manor Heights', 'Sunnyside',
       'Concord', 'New Brighton', 'Sandy Ground', 'Randall Manor',
       'Richmond Town', 'Grymes Hill', 'Richmond Valley', 'Port Ivory',
       'Lighthouse Hill', 'Huguenot', "Mariner's Harbor", 'Grasmere',
       'West Brighton', 'Butler Manor', 'Tottenville', 'Travis',
       'Westerleigh', 'Arlington', 'Oakwood', 'Bloomfield', 'Bay Terrace',
       'Tompkinsville', 'Chelsea', 'Charleston', 'Todt Hill'],
      dtype=object)

In [35]:
cluster_1["Neighborhood"].unique()

array(['Old Town', 'New Springville', 'Pleasant Plains', 'St. George',
       'Willowbrook', 'Stapleton', 'Woodrow', 'Rosebank', 'Dongan Hills',
       'Clifton', 'Elm Park', 'Eltingville', 'Arrochar', 'Bulls Head',
       'Heartland Village', 'Great Kills', 'New Dorp', 'Grant City'],
      dtype=object)

In [36]:
cluster_2["Neighborhood"].unique()

array(['Arden Heights', 'Annadale', 'Castleton Corners', 'Port Richmond',
       'Greenridge', 'Rossville', "Prince's Bay"], dtype=object)

# Observations

Most of the Pizza Places are in cluster 2 which are around Tompkinsville, Rosebank, Shore Acres, etc. 
Lowest Pizza Places are present around neighborhoods in Cluster 0. There are many neighborhoods in Cluster 0. Though most of these neighborhoods are located near the neighborhoods in Cluster 2 and Cluster 1. So to start a Pizza Place in such location is not a good choice. The neighborhoods around Tottenville, Howland Hook, Travis, West Brighton, Sunnyside are some of the optimal locations to open a pizza place. 